In [5]:
from sentence_transformers import SentenceTransformer
from gnd_dataset import GNDDataset
from gnd_graph import GNDGraph
from data_collator import DataCollator
import yaml
from tqdm import tqdm
import os
import torch
from torch.utils.data import DataLoader
from torch import nn
from datasets import Dataset
import pickle
from retriever import Retriever
import faiss
import pandas as pd

from ast import literal_eval
from utils import  load_model
from safetensors import safe_open

from statistics import mean, stdev

In [6]:
gnd_path = "data/gnd.pickle"
config_path = "configs_small/dev.yaml"
# Load config 
with open(config_path, "r") as f:
    config = yaml.safe_load(f)
gnd_graph = pickle.load(open(gnd_path, "rb"))
gnd_graph = GNDGraph(gnd_graph)

path = "pt_models/temp-testing/best_model/model.safetensors"
model, tokenizer = load_model(path, config=config, device="cuda:0")

In [7]:
ds = GNDDataset("datasets/no_context", 
                gnd_graph=gnd_graph, 
                config=config, 
                load_from_disk=True,)
ds = ds["test"]
collator = DataCollator( 
    tokenizer=tokenizer, 
    graph=gnd_graph,
    config=config,
    device="cuda:0")
dataloader = DataLoader(
    ds, 
    batch_size=1, 
    collate_fn= lambda x: collator(x, inference=True),
    shuffle=False,
    num_workers=0,
)

In [8]:
model = model.to("cuda:0")
model.eval()
c = 0
for i in dataloader:
    c += 1
    if c >= 20:
        break
    with torch.no_grad():
        print(i["title"])
        i = {k: v.to("cuda:0").unsqueeze(0) for k, v in i.items() if isinstance(v, torch.Tensor)}
        print(tokenizer.decode(i["input_ids"][0], skip_special_tokens=True))
        outputs = model.module.generate(
            **i,
            do_sample=False,
            temperature=None,
            top_p=None,
            num_beams=1,
        )
        
        length_input = i["input_ids"].shape[1]
        tokens = outputs[:, length_input:]
        tokens = tokenizer.batch_decode(tokens, skip_special_tokens=True)
        print(tokens[0])
        print("---"*30)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Die Landesministerkonferenzen und der Bund kooperativer Föderalismus im Schatten der Politikverflechtung
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Die Landesministerkonferenzen und der Bund kooperativer Föderalismus im Schatten der Politikverflechtung. Schlagwörter: 
Debroughton-
------------------------------------------------------------------------------------------
Die Geburt der Philosophie im Garten der Lüste Michel Foucaults Archäologie des platonischen Eros
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Die Geburt der Philosophie im Garten der Lüste Michel Foucaults Archäologie des platonischen Eros. Schlagwör

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Philosophy
------------------------------------------------------------------------------------------
Das Geldwäscherisiko verschiedener Glücksspielarten
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Das Geldwäscherisiko verschiedener Glücksspielarten. Schlagwörter: 


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Pattaching (Berefifty (Berefifty (Berefifty (B
------------------------------------------------------------------------------------------
Bienen gesund erhalten Krankheiten vorbeugen, erkennen und behandeln
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Bienen gesund erhalten Krankheiten vorbeugen, erkennen und behandeln. Schlagwörter: 


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Berefifty (Berefifty (Berefifty (Berefifty (
------------------------------------------------------------------------------------------
Blitzrezepte für Hundekekse Gesunde Leckereien selber backen
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Blitzrezepte für Hundekekse Gesunde Leckereien selber backen. Schlagwörter: 


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Helenkevin (Berefifty (Berefifty (Berefifty (
------------------------------------------------------------------------------------------
Igel sucht Unterschlupf So helfe ich Tieren über den Winter
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Igel sucht Unterschlupf So helfe ich Tieren über den Winter. Schlagwörter: 
I'my
------------------------------------------------------------------------------------------
Bewässerung im Garten- und Landschaftsbau
Du bist ein hilfreicher Assistent für Bibliothekare. Du hilfst bei der Katalogisierung von Büchern. Beispiel: Titel: Die Entdeckung der Langsamkeit. Schlagwörter: Entdeckung; Langsamkeit; Natur; Wissenschaft; Philosophie. Titel: Bewässerung im Garten- und Landschaftsbau. Schlagwörter: 
Gewa (Berefifty (Berefifty (Berefifty (B
----------------------------

In [24]:
results = [
    "results/prompt-tuning-baseline-full",
    "results/prompt-tuning-from_gnd",
    "results/pt-from_gnd_w_alt",
]

dfs = []
analysis = {
}
for result in results:
    files = os.listdir(result)
    for file in files:
        if file.endswith(".csv"):
            data_path = os.path.join(result, file)
            df = pd.read_csv(data_path)
            df["predictions"] = df["predictions"].apply(literal_eval)
            df["label-ids"] = df["label-ids"].apply(literal_eval)
            analysis["gold-labels"] = df["label-ids"]
            exp_name = os.path.basename(result)
            analysis[f"{exp_name}-predictions"] = df["predictions"]
            analysis["title"] = df["title"]
            

In [34]:
analysis_df = pd.DataFrame.from_dict(analysis)

# Union of all predictions
analysis_df["union-predictions"] = analysis_df.apply(
    lambda row: set(idn for column in analysis_df.columns for idn in row[column] if column != "title"),
    axis=1
)

In [35]:
analysis_df["overall-num"] = analysis_df["union-predictions"].apply(lambda x: len(x))
sorted_df = analysis_df.sort_values(by="overall-num", ascending=False)

In [36]:
sorted_df.head(20)

,gold-labels,prompt-tuning-baseline-full-predictions,title,prompt-tuning-from_gnd-predictions,pt-from_gnd_w_alt-predictions,union-predictions,overall-num
2266,"[040036146, 040036774, 040152170, 040152723, 0...","[040775879, 944275982]",Vergangene Vertrautheit Soziale Gedächtnisse ...,"[042007933, 040139603, 957627521]","[042895189, 040775704]","{042525799, 040778142, 040775704, 042518717, 0...",23
2960,"[004691792, 040267768, 040465748, 040511766, 0...","[040775755, 040139603, 118749404, 040511766, 0...",Katholische Arbeiter im Mutterland der Reforma...,"[040511766, 040475387, 004691792, 041207300, 0...","[040475387, 040511766, 040300277, 041207300, 0...","{041142047, 004691792, 041402928, 040465748, 0...",22
7130,"[040163199, 040169162, 040175553, 04060568X, 0...","[040351653, 040322319, 956910688, 040063402, 0...",Konzeptionierung und Auslegung eines Vorderach...,"[041263928, 040168999, 95852923X, 040760014, 0...","[040351653, 04139402X, 948677694, 040168999, 0...","{960526005, 040063402, 041263928, 040169162, 9...",22
2142,"[040142388, 040316483, 040550729, 040760014, 0...","[040737578, 946945187, 04318037X]",Verlustmechanismen in einem halbhermetischen P...,"[040383903, 040663876, 1014172047, 041277988, ...","[042090377, 041878795, 043080588, 041396294, 0...","{040737578, 957807074, 946945187, 94910938X, 0...",22
3891,"[040063496, 040316300, 040476863, 04060294X, 0...","[04040949X, 040402355, 040623866, 042908027, 9...","Je dunkler die Nacht, desto heller die Sterne ...","[040703347, 040152723, 041583132, 040039005, 9...","[042908027, 040602907, 043283349, 040623866, 0...","{041893999, 040063496, 040348695, 042908027, 0...",21
5553,"[040409368, 040729052, 041209575, 041417666, 0...","[040715612, 041451430, 042852218, 040386392]",Beiträge zur KI-basierten Erkennung menschlic...,"[041289870, 041209575, 040063119, 040334473]","[040521540, 041451317, 043204864, 041571673]","{041970322, 040409368, 043144403, 041451317, 0...",21
7257,"[001173464, 001213571, 004815262, 004816730, 0...","[040488217, 040457915, 04059758X, 118749404, 0...",Späthumanismus und reformierte Konfession The...,"[04059758X, 040386392, 040457915, 040239969, 0...","[040261409, 041772342, 04059758X, 040488217, 0...","{041363434, 001213571, 004816730, 041649486, 0...",21
7427,"[040312178, 040441962, 040626563, 040642119, 0...","[1041356269, 041281306, 041864700]",Experimenteller Beitrag zur Thermodynamik im F...,"[040580768, 041281306, 041273478, 04148259X, 0...","[043161324, 041246268, 040641767]","{040642119, 041281306, 041243005, 04148259X, 0...",21
4791,"[040382435, 040418626, 040457915, 040492508, 0...","[041361482, 04010074X, 1007668032, 040687546, ...",Katharsis in Kaiserzeit und Spätantike Vorste...,[041401972],"[041779665, 1067343555, 952942186, 041361482, ...","{118595164, 040463036, 041361482, 040457915, 1...",21
4625,"[040322726, 040348377, 04046170X, 040646890, 0...","[041513800, 041904966, 041833678, 041328256]",Einfluss von Wasser in Getriebeölen auf die Z...,"[042090377, 041528786, 040673332, 041209974, 9...","[041513800, 041563077, 040529223, 041833678]","{040529223, 041308263, 04046170X, 040646890, 0...",20


In [3]:
result_path = "results/prompt-tuning-baseline-full"
files = os.listdir(result_path)
all_metrics = {
    "recall": [],
    "precision": [],
    "f1": [],
    "weighted_precision": [],
}
for f in files:
    path = os.path.join(result_path, f)
    if not f.endswith(".yaml"):
        continue
    with open(path, "r") as file:
        metrics = yaml.safe_load(file)
    for key in all_metrics:
        if key in metrics:
            all_metrics[key].append(100*metrics[key])


print("Recall: {:.2f} ± {:.2f}".format(mean(all_metrics["recall"]), stdev(all_metrics["recall"])))
print("Precision: {:.2f} ± {:.2f}".format(mean(all_metrics["precision"]), stdev(all_metrics["precision"])))
print("F1: {:.2f} ± {:.2f}".format(mean(all_metrics["f1"]), stdev(all_metrics["f1"])))
print("Weighted precision: {:.2f} ± {:.2f}".format(mean(all_metrics["weighted_precision"]), stdev(all_metrics["weighted_precision"])))



Recall: 37.69 ± 0.22
Precision: 45.88 ± 0.26
F1: 39.04 ± 0.20
Weighted precision: 50.72 ± 0.27


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
path = os.path.join("models_gnd/prompt-tuning-gnd-neighbor2label-nohidden/best_model/model.safetensors")
keys_to_load = config.get("load_pt_keys", None)
print(keys_to_load)

model, tokenizer = load_model(path, config, device=device, load=keys_to_load)

['prompt_generator.prompt.weight', 'model.embed_tokens.weight']


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.18it/s]


Loading tensor model.embed_tokens.weight from checkpoint.
Loading tensor model.prompt_generator.prompt.weight from checkpoint.


In [7]:
from huggingface_hub import HfApi
api = HfApi()
path = "models_gnd/pt-from_gnd_w_alt/best_model/"
repo = "KatjaK/LLama-Prompt-Tuning-finetuned-from-Neighbor-and-Altnames2Label-GND"
# api.create_repo(
#     repo_id=repo
# )
api.upload_folder(
    folder_path=path,
    repo_id=repo,
    repo_type="model",
)

model.safetensors:   0%|          | 0.00/1.60G [00:00<?, ?B/s]


scheduler.pt: 100%|██████████| 1.40k/1.40k [00:00<00:00, 3.05kB/s]B/s]
model.safetensors: 100%|██████████| 1.60G/1.60G [00:46<00:00, 34.6MB/s]

optimizer.pt: 100%|██████████| 3.20G/3.20G [01:27<00:00, 36.6MB/s]

Upload 3 LFS files: 100%|██████████| 3/3 [01:27<00:00, 29.27s/it]


CommitInfo(commit_url='https://huggingface.co/KatjaK/LLama-Prompt-Tuning-finetuned-from-Neighbor-and-Altnames2Label-GND/commit/63030b4737e41722061d29ba01148b62a6bf3cd4', commit_message='Upload folder using huggingface_hub', commit_description='', oid='63030b4737e41722061d29ba01148b62a6bf3cd4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KatjaK/LLama-Prompt-Tuning-finetuned-from-Neighbor-and-Altnames2Label-GND', endpoint='https://huggingface.co', repo_type='model', repo_id='KatjaK/LLama-Prompt-Tuning-finetuned-from-Neighbor-and-Altnames2Label-GND'), pr_revision=None, pr_num=None)